In [20]:
import os
import sys
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import argparse
from sklearn.metrics import roc_auc_score, average_precision_score
sys.path.insert(0,'/data/vijay/git/pancancer/scripts/util')
print(sys.path)
#from data.vijay.git.pancancer.scripts.util.tcga_util import get_args
from tcga_util import get_args

['/data/vijay/git/pancancer/scripts/util', '/data/vijay/git/pancancer/scripts/util', '', '/home/nagampv/anaconda3/lib/python36.zip', '/home/nagampv/anaconda3/lib/python3.6', '/home/nagampv/anaconda3/lib/python3.6/lib-dynload', '/home/nagampv/anaconda3/lib/python3.6/site-packages', '/home/nagampv/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/home/nagampv/.ipython']


In [21]:
# Get the current working directory
cwd = os.getcwd()
print(cwd)
# Ensure that the path is starting in the scripts directory
if not cwd.split('/')[-1] == 'scripts':
    sys.path.append(os.path.join(cwd, 'scripts'))

/data/vijay/git/pancancer/scripts


In [3]:
def get_gene_auroc(x, w):
    score = roc_auc_score(x, w, average='weighted')
    return(score)

def get_gene_auprc(x, w):
    score = average_precision_score(x, w, average='weighted')
    return(score)

In [4]:
#args = get_args()
#alt_folder = args.alt_folder
#abc_folder = alt_folder

abc_folder = os.path.join('..', 'classifiers', 'PI3K_GAIN_MOD')

In [5]:
# Load Datasets
mut_file = os.path.join('..', 'data', 'pancan_mutation_freeze.tsv')
sample_freeze_file = os.path.join('..', 'data', 'sample_freeze.tsv')
copy_loss_file = os.path.join('..', 'data', 'copy_number_loss_status.tsv')
copy_gain_file = os.path.join('..', 'data', 'copy_number_gain_status.tsv')

mutation_df = pd.read_table(mut_file, index_col=0)
sample_freeze = pd.read_table(sample_freeze_file, index_col=0)
copy_loss_df = pd.read_table(copy_loss_file, index_col=0)
copy_gain_df = pd.read_table(copy_gain_file, index_col=0)

In [6]:
# Load pi3k Pathway Genes
pi3k_genes_file = os.path.join('..', 'data', 'pi3k_genes.csv')
pi3k_genes_df = pd.read_csv(pi3k_genes_file)
pi3k_genes_df.head()

,genes,og_tsg
0,PIK3CA,OG
1,PIK3CB,OG
2,PIK3R2,OG
3,AKT1,OG
4,AKT2,OG


In [7]:
# Load classifier weights
pi3k_decision_file = os.path.join(abc_folder, 'classifier_decisions.tsv')
pi3k_decisions_df = pd.read_table(pi3k_decision_file)
pi3k_decisions_df.head()

,SAMPLE_BARCODE,log10_mut,total_status,weight,ERBB2,PIK3CA,AKT1,KRAS,AKT1_gain,ERBB2_gain,KRAS_gain,PIK3CA_gain,PATIENT_BARCODE,DISEASE,SUBTYPE,hypermutated,include
0,TCGA-02-0047-01,1.812913,1,0.442756,0,1,0,0,0,0,0,0,TCGA-02-0047,GBM,IDHwt,0,0
1,TCGA-02-0055-01,1.707570,0,0.241695,0,0,0,0,0,0,0,0,TCGA-02-0055,GBM,IDHwt,0,0
2,TCGA-02-2483-01,1.662758,0,0.438278,0,0,0,0,0,0,0,0,TCGA-02-2483,GBM,IDHmut-non-codel,0,0
3,TCGA-02-2485-01,1.748188,1,0.480901,0,1,0,0,0,0,0,0,TCGA-02-2485,GBM,IDHwt,0,0
4,TCGA-02-2486-01,1.755875,0,0.388514,0,0,0,0,0,0,0,0,TCGA-02-2486,GBM,IDHwt,0,0


In [8]:
pi3k_mutations_df = mutation_df[pi3k_genes_df['genes']]

# Add status to the Y matrix depending on if the gene is a tumor suppressor
# or an oncogene. An oncogene can be activated with copy number gains, but
# a tumor suppressor is inactivated with copy number loss

oncogene = pi3k_genes_df[pi3k_genes_df['og_tsg'] == 'OG']
tumor_suppressor = pi3k_genes_df[pi3k_genes_df['og_tsg'] == 'TSG']

# Subset copy number information
pi3k_copy_gain_sub_df = copy_gain_df[oncogene['genes']]
pi3k_copy_loss_sub_df = copy_loss_df[tumor_suppressor['genes']]

# Combine Copy Number data
pi3k_copy_df = pd.concat([pi3k_copy_gain_sub_df, pi3k_copy_loss_sub_df], axis=1)

In [9]:
pi3k_status_df = pi3k_mutations_df + pi3k_copy_df
pi3k_status_df[pi3k_status_df == 2] = 1
pi3k_status_df.head()

,PIK3CA,PIK3CB,PIK3R2,AKT1,AKT2,AKT3,RHEB,RPTOR,RICTOR,MTOR,...,NFE2L2,PTEN,PIK3R1,PIK3R3,INPP4B,STK11,PPP2R1A,TSC1,TSC2,NF1
SAMPLE_BARCODE,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047-01,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-0055-01,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
TCGA-02-2483-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-2485-01,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-2486-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
subset_columns = ['SAMPLE_BARCODE', 'DISEASE', 'weight', 'total_status', 'log10_mut',
                  'hypermutated', 'include']
pi3k_decisions_subset_df = pi3k_decisions_df[subset_columns]
pi3k_full_status_df = pi3k_status_df.merge(pi3k_decisions_subset_df, left_index=True,
                                         right_on='SAMPLE_BARCODE')
pi3k_full_status_df.index = pi3k_full_status_df['SAMPLE_BARCODE']
pi3k_full_status_df.head()

,PIK3CA,PIK3CB,PIK3R2,AKT1,AKT2,AKT3,RHEB,RPTOR,RICTOR,MTOR,...,TSC1,TSC2,NF1,SAMPLE_BARCODE,DISEASE,weight,total_status,log10_mut,hypermutated,include
SAMPLE_BARCODE,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047-01,1,0,0,0,0,0,0,0,0,0,...,0,0,0,TCGA-02-0047-01,GBM,0.442756,1,1.812913,0,0
TCGA-02-0055-01,0,0,0,0,0,0,0,0,0,0,...,0,0,1,TCGA-02-0055-01,GBM,0.241695,0,1.707570,0,0
TCGA-02-2483-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,TCGA-02-2483-01,GBM,0.438278,0,1.662758,0,0
TCGA-02-2485-01,1,0,0,0,0,1,0,0,0,0,...,0,0,0,TCGA-02-2485-01,GBM,0.480901,1,1.748188,0,0
TCGA-02-2486-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,TCGA-02-2486-01,GBM,0.388514,0,1.755875,0,0


In [18]:
# Remove hyper mutated samples
burden_filter = pi3k_full_status_df['hypermutated'] == 0
burden_filter = burden_filter & pi3k_full_status_df['log10_mut'] < 5 * pi3k_full_status_df['log10_mut'].std()
pi3k_full_status_df = pi3k_full_status_df[burden_filter]
pi3k_full_status_df.head(3)
pi3k_full_status_df.shape

(9074, 31)

In [12]:
full_auroc = (
    pi3k_full_status_df[pi3k_genes_df['genes']]
    .apply(lambda x: get_gene_auroc(x, pi3k_full_status_df['weight']))
    )
full_auprc = (
    pi3k_full_status_df[pi3k_genes_df['genes']]
    .apply(lambda x: get_gene_auprc(x, pi3k_full_status_df['weight']))
    )

In [19]:
# Remove pi3k positive samples, and recalculate metrics
#drop abc genes:
genes_df = pd.read_csv("/data/vijay/git/pancancer/data/abc_genes.csv")
genes = genes_df['genes'].tolist()
print(genes)
remove_pi3k_status = pi3k_full_status_df[pi3k_full_status_df['total_status'] == 0]
remove_pi3k_status_df = remove_pi3k_status[pi3k_genes_df['genes']]
remove_pi3k_status_df = remove_pi3k_status_df.drop(genes, axis=1)
remove_pi3k_status_df.shape
#full_auroc_remove = remove_pi3k_status_df.apply(lambda x: get_gene_auroc(x, w=remove_pi3k_status['weight']))
#full_auprc_remove = remove_pi3k_status_df.apply(lambda x: get_gene_auprc(x, w=remove_pi3k_status['weight']))

['AKT1', 'PIK3CA', 'PIK3R1', 'PTEN', 'STK11', 'INPP4B']


(6273, 18)

In [16]:
# Get output metrics for pi3k classification
output_pi3k_metrics = pd.concat([full_auroc, full_auroc_remove], axis=1, sort=False)
output_pi3k_metrics = output_pi3k_metrics * 100  # To get percent
output_pi3k_metrics = output_pi3k_metrics - 50  # Subtract 50 from AUROC only

# Combine with AUPRC
output_pi3k_metrics = pd.concat([output_pi3k_metrics, full_auprc * 100,
                                full_auprc_remove * 100], axis=1, sort=False)
output_pi3k_metrics.columns = ['pi3k_auroc', 'no_pi3k_auroc', 'pi3k_auprc', 'no_pi3k_auprc']

# Fill removed pi3k metrics with included metrics
output_pi3k_metrics['no_pi3k_auroc'] = (
    output_pi3k_metrics['no_pi3k_auroc'].fillna(output_pi3k_metrics['pi3k_auroc'])
    )
output_pi3k_metrics['no_pi3k_auprc'] = (
    output_pi3k_metrics['no_pi3k_auprc'].fillna(output_pi3k_metrics['pi3k_auprc'])
    )

# Write results to file
tables_folder = os.path.join(abc_folder, 'tables')

if not os.path.exists(tables_folder):
    os.makedirs(tables_folder)

pi3k_metric_file = os.path.join(abc_folder, 'tables', 'pi3k_metrics_pathwaymapper.txt')
output_pi3k_metrics.to_csv(pi3k_metric_file, sep='\t')

output_pi3k_metrics.head()

,pi3k_auroc,no_pi3k_auroc,pi3k_auprc,no_pi3k_auprc
PIK3CA,11.538193,11.538193,29.707709,29.707709
PIK3CB,13.292059,6.643529,6.049201,3.219805
PIK3R2,7.091626,3.396380,2.922285,2.030019
AKT1,11.170993,11.170993,3.011951,3.011951
AKT2,8.158848,6.618688,3.295087,2.799736


In [17]:
# Display pi3k pathway metrics
all_samples_pi3k_pathway_status = pi3k_full_status_df[pi3k_genes_df['genes']].max(axis=1)
print('pi3k Pathway Performance Summary: All pi3k Genes')
print('AUROC:')
print(roc_auc_score(all_samples_pi3k_pathway_status,
                    pi3k_full_status_df['weight'], average='weighted'))
print('AUPRC:')
print(average_precision_score(all_samples_pi3k_pathway_status,
                              pi3k_full_status_df['weight'], average='weighted'))

pi3k Pathway Performance Summary: All pi3k Genes
AUROC:
0.6382545864112827
AUPRC:
0.7275336003463387


In [18]:
print('pi3k Pathway Performance Summary:', genes)
print('AUROC:')
print(roc_auc_score(pi3k_full_status_df['total_status'],
                    pi3k_full_status_df['weight'], average='weighted'))
print('AUPRC:')
print(average_precision_score(pi3k_full_status_df['total_status'],
                              pi3k_full_status_df['weight'], average='weighted'))

pi3k Pathway Performance Summary: ['AKT1', 'PIK3CA', 'PIK3R1', 'PTEN', 'STK11', 'INPP4B']
AUROC:
0.8199567751744343
AUPRC:
0.6233651115489887


In [19]:
print('pi3k Pathway Performance Summary: Held Out Samples')
held_out_pi3k_df = pi3k_full_status_df[pi3k_full_status_df['include'] == 0]
print('AUROC:')
print(roc_auc_score(held_out_pi3k_df['total_status'],
                    held_out_pi3k_df['weight'], average='weighted'))
print('AUPRC:')
print(average_precision_score(held_out_pi3k_df['total_status'],
                              held_out_pi3k_df['weight'], average='weighted'))

pi3k Pathway Performance Summary: Held Out Samples
AUROC:
0.7051315768627513
AUPRC:
0.37883676768463836


# Visualize Distribution of AUROC and AUPRC for all genes

In [20]:
# Subset mutation file by samples
sub_full_mutation_df = mutation_df[burden_filter]
low_mutation_count_filter = (
    sub_full_mutation_df.sum()
    [sub_full_mutation_df.sum() >= 10].sort_values(ascending=False).index
    )
sub_full_mutation_df = sub_full_mutation_df[low_mutation_count_filter]
sub_full_mutation_df.head()

,TP53,TTN,MUC16,PIK3CA,CSMD3,RYR2,LRP1B,SYNE1,FLG,USH2A,...,C17orf50,C17orf105,C17orf103,MBD3L2,DKFZP761K2322,RP11-434D2.3,FXYD4,AC110619.2,RP11-386M24.4,C15orf40
SAMPLE_BARCODE,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047-01,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-0055-01,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-2483-01,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-2485-01,1,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-2486-01,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Get Metrics for All Genes
all_auprc = sub_full_mutation_df.apply(lambda x: get_gene_auprc(x, w = pi3k_full_status_df['weight']))
all_auroc = sub_full_mutation_df.apply(lambda x: get_gene_auroc(x, w = pi3k_full_status_df['weight']))

In [23]:
# Process file and save results
all_gene_metrics_file = os.path.join(abc_folder, 'tables', 'all_gene_metric_ranks.tsv')

all_genes_auprc_df = pd.DataFrame(all_auprc.sort_values(ascending=False), columns=['auprc'])
all_genes_auroc_df = pd.DataFrame(all_auroc.sort_values(ascending=False), columns=['auroc'])

all_genes_auprc_df = all_genes_auprc_df.assign(auprc_rank = list(range(0, all_genes_auprc_df.shape[0])))
all_genes_auroc_df = all_genes_auroc_df.assign(auroc_rank = list(range(0, all_genes_auprc_df.shape[0])))

all_genes_auprc_df = all_genes_auprc_df.assign(pi3k = 0)
all_genes_auprc_df.loc[all_genes_auprc_df.index.isin(pi3k_genes_df['genes']), 'pi3k'] = 1

all_genes_metrics_df = all_genes_auprc_df.reset_index().merge(all_genes_auroc_df,
                                                              left_on='index', right_index=True)

all_genes_metrics_df.columns = ['Gene', 'AUPRC', 'AUPRC Rank', 'pi3k', 'AUROC', 'AUROC Rank']
all_genes_metrics_df.to_csv(all_gene_metrics_file, sep='\t', index=False)
all_genes_metrics_df.head(10)

,Gene,AUPRC,AUPRC Rank,pi3k,AUROC,AUROC Rank
0,PTEN,0.588693,0,1,0.871275,74
1,TP53,0.368805,1,0,0.509373,18960
2,TTN,0.354309,2,0,0.557267,17178
3,PIK3CA,0.263492,3,1,0.630881,6932
4,ARID1A,0.252372,4,0,0.664979,3117
5,PIK3R1,0.227745,5,1,0.770791,343
6,MUC16,0.222872,6,0,0.530100,18543
7,RYR2,0.171702,7,0,0.580962,14726
8,CTCF,0.171120,8,0,0.733214,645
9,CSMD3,0.170909,9,0,0.565058,16510
